In [1]:
import os, sys
import numpy as np
import json
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, Adam
import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Preliminary

In [2]:
torch.manual_seed(12345)
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')

na_list = ['A', 'C', 'G', 'T'] #nucleic acids
aa_list = ['R', 'L', 'S', 'A', 'G', 'P', 'T', 'V', 'N', 'D', 'C', 'Q', 'E', 'H', 'I', 'K', 'M', 'F', 'W', 'Y'] #amino acids
NNK_freq = [0.09375]*3 + [0.0625]*5 + [0.03125]*13 #freq of 21 NNK codons including the stop codon
sum_20 = 0.0625*5 + 0.09375*3 + 0.03125*12 #sum of freq without the stop codon
pvals = [0.09375/sum_20]*3 + [0.0625/sum_20]*5 + [0.03125/sum_20]*12 #normalize freq for 20 codons
pvals = [0.09375/sum_20]*3 + [0.0625/sum_20]*5 + [0.03125/sum_20]*11 + \
        [1- sum([0.09375/sum_20]*3 + [0.0625/sum_20]*5 + [0.03125/sum_20]*11)] 
        #adjust sum to 1 due to numerical issue
aa_dict = dict(zip(aa_list, pvals))

## Dataset

In [3]:
def construct_dataset():
    with open(aptamer_dataset_file, 'r') as f:
        aptamer_data = json.load(f)
    full_dataset = []
    for aptamer in aptamer_data:
        peptides = aptamer_data[aptamer]
        if aptamer == "CTTTGTAATTGGTTCTGAGTTCCGTTGTGGGAGGAACATG": #took out aptamer control
            continue
        for peptide, _ in peptides:
            peptide = peptide.replace("_", "") #removed stop codons
            if "RRRRRR" in peptide: #took out peptide control
                continue
            if len(aptamer) == 40 and len(peptide) == 8: #making sure right length
                full_dataset.append((aptamer, peptide))
    full_dataset = list(set(full_dataset)) #removed duplicates
    return full_dataset

class TrainDataset(torch.utils.data.Dataset):
    def __init__(self, training_set):
        super(TrainDataset, self).__init__() 
        self.training_set = training_set
        
    def __len__(self):
        return len(self.training_set)

    def __getitem__(self, idx):
        aptamer, peptide = self.training_set[idx]
        return aptamer, peptide
    
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, test_set):
        super(TestDataset, self).__init__() 
        self.test_set = test_set
        
    def __len__(self):
        return len(self.test_set)

    def __getitem__(self, idx):
        aptamer, peptide = self.test_set[idx]
        return aptamer, peptide

In [4]:
aptamer_dataset_file = "../data/aptamer_dataset.json"
full_dataset = construct_dataset()
n = len(full_dataset)
training_set = full_dataset[:int(0.8*n)]
test_set = full_dataset[int(0.8*n):]
train_dataset = TrainDataset(training_set)
test_dataset = TestDataset(test_set)
train_loader = torch.utils.data.DataLoader(train_dataset)
test_loader = torch.utils.data.DataLoader(test_dataset)

## NN Models

In [5]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        super(SimpleConvNet, self).__init__()
        self.cnn_apt_1 = nn.Conv2d(1, 5, (4,4)) #similar to 3-gram
        self.cnn_pep_1 = nn.Conv2d(1, 5, (4,20))
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(210, 1)
        
    def forward(self, apt, pep):
        apt = self.cnn_apt_1(apt)
        apt = self.relu(apt)
        pep = self.cnn_pep_1(pep)
        pep = self.relu(pep)
        apt = apt.view(-1, 1).T
        pep = pep.view(-1, 1).T
        
        x = torch.cat((apt, pep), 1)
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        nn.init.zeros_(m.bias.data)
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
        nn.init.zeros_(m.bias.data)

## Sampling

In [6]:
# Sample x from P_X (assume apatamers follow uniform)
def get_x():
    x_idx = np.random.randint(0, 4, 40)
    x = ""
    for i in x_idx:
        x += na_list[i]
    return x

# Sample y from P_y (assume peptides follow NNK)
def get_y():
    y_idx = np.random.choice(20, 7, p=pvals)
    y = "M"
    for i in y_idx:
        y += aa_list[i]
    return y

# Generate uniformly from S without replacement
def get_xy(k):
    samples = [full_dataset[i] for i in np.random.choice(len(full_dataset), k, replace=False)]
    return samples

# S' contains S with double the size of S (domain for Importance Sampling)
def get_S_prime(k):
    S_prime_dict = dict.fromkeys(full_dataset, 0) #indicator 0 means in the original dataset
    S_new = []
    for _ in range(k):
        pair = (get_x(), get_y())
        S_prime_dict[pair] = 1 #indicator 1 means not in the original dataset
        S_new.append(pair)
    S_prime = [[k,int(v)] for k,v in S_prime_dict.items()]
    random.shuffle(S_prime)
    return S_prime, S_new

# Returns pmf of an aptamer
def get_x_pmf():
    return 0.25**40

# Returns pmf of a peptide
def get_y_pmf(y):
    pmf = 1
    for char in y[1:]: #skips first char "M"
        pmf *= aa_dict[char]
    return pmf

S_prime, S_new = get_S_prime(n) #use for sgd and eval

## SGD

In [27]:
## Takes a peptide and aptamer sequence and converts to one-hot matrix
def one_hot(sequence, seq_type='peptide'):
    if seq_type == 'peptide':
        letters = aa_list
    else:
        letters = na_list
    one_hot = np.zeros((len(sequence), len(letters)))
    for i in range(len(sequence)):
        char = sequence[i]
        for _ in range(len(letters)):
            idx = letters.index(char)
            one_hot[i][idx] = 1
    return one_hot

# Convert a pair to one-hot tensor
def convert(apt, pep): 
    apt = one_hot(apt, seq_type='aptamer') #(40, 4)
    pep = one_hot(pep, seq_type='peptide') #(8, 20)
    apt = torch.FloatTensor(np.reshape(apt, (1, 1, apt.shape[0], apt.shape[1]))).cuda() #(1, 1, 40, 4)
    pep = torch.FloatTensor(np.reshape(pep, (1, 1, pep.shape[0], pep.shape[1]))).cuda() #(1, 1, 8, 20)
    return apt, pep

def update(x, y):
    pmf = get_y_pmf(y)
    x.requires_grad=True
    y.requires_grad=True
    x = x.cuda()
    y = y.cuda()
    out = model(x, y)
    return pmf, out

def get_log_out(k):
    outs = []
    for (apt, pep) in full_dataset[:k]:
        x, y = convert(apt, pep)
        _, out = update(x, y)
        outs.append(torch.log(out).cpu().detach().numpy().flatten()[0])
    return np.average(outs)

def get_out_prime(k):
    outs = []
    for (apt, pep) in S_new[:k]:
        x, y = convert(apt, pep)
        _, out = update(x, y)
        outs.append(out.cpu().detach().numpy().flatten()[0])
    return np.average(outs)
    
def apply_param_grad(grads1, grads2, fn):
    gs = []
    for grad1, grad2 in zip(grads1, grads2):
        gs.append(fn(grad1, grad2))
    return gs

def sgd(t=1, #num of iter over the training set
        lamb=1e-1, #hyperparam
        gamma=1e-2): #step size
    optim = SGD(model.parameters(), lr=gamma)
    model.train()
    for _ in range(t):
        for i, (apt, pep) in enumerate(tqdm.tqdm(train_loader)):
            optim.zero_grad()
            x, y = convert(apt[0], pep[0])
            _, out = update(x, y)
            log_out = torch.log(out)
#             log_out.backward(retain_graph=True)
#             g1 = []
#             for param in model.parameters():
#                 g1.append(param.grad)

            optim.zero_grad()
            x_prime, y_prime = convert(S_prime[i][0][0], S_prime[i][0][1])
            y_pmf, out_prime = update(x_prime, y_prime)
            out_prime = out_prime*y_pmf*get_x_pmf()*2*n
#             out_prime.backward(retain_graph=True)
#             g2 = []
#             for param in model.parameters():
#                 g2.append(param.grad)

            const = S_prime[i][1] #indicator
#             gs = apply_param_grad(g1, g2, lambda g1, g2: lamb*const*g2 - g1)
#             for param, g in zip(model.parameters(), gs): #update params
#                 param.grad = g

            (lamb*const*out_prime - log_out).backward(retain_graph=True)
            optim.step()
            if i % 500 == 0:
                loss = lamb*get_out_prime(1000) - get_log_out(1000)
                print("Iteration: ", i, " Loss: %.3f" % loss)

In [8]:
# Recall on train set of size k to test for overfitting
def recall_train(k):
    model.eval()
    correct = 0
    train_recall_outputs = []
    for (apt, pep) in recall_train_samples:
        apt, pep = convert(apt, pep)
        out = model(apt, pep).cpu().detach().numpy().flatten()[0]
        train_recall_outputs.append(out)
        if out > 0.75:
            correct += 1
    train_recall = 100*correct/k #recall rate of k samples from training set
    return train_recall, train_recall_outputs #list of k outputs


# Recall on test set of size k
def recall_test(k):
    model.eval()
    correct = 0
    test_recall_outputs = []
    for i, (aptamer, peptide) in enumerate(tqdm.tqdm(test_loader)):
        if i > k:
            break
        apt, pep = convert(aptamer[0], peptide[0])
        output = model(apt, pep).cpu().detach().numpy().flatten()[0]
        test_recall_outputs.append(output)
        if output > 0.75:
            correct += 1
    test_recall = 100*correct/k #recall rate of k samples from test set
    return test_recall, test_recall_outputs #list of k outputs


# Eval on m new unseen pairs in S_new (not in our dataset)
def eval_unknown(m):
    model.eval()
    eval_unknown_outputs = []
    for i, (x, y) in enumerate(S_new[:m]):
        apt, pep = convert(x, y)
        output = model(apt, pep).cpu().detach().numpy().flatten()[0]
        eval_unknown_outputs.append(output)
    return eval_unknown_outputs #list of m outputs


# AUC Plot
def cdf(scores, i): # i is the index
    plt.hist(scores, 100, histtype='step', density=True, cumulative=True)
    g = gammas[i//len(lambdas)]
    l = lambdas[i%len(lambdas)]
    label = 'lambda =%.5f' % l  + ' gamma =%.5f' % g
    plt.legend([label])
    plt.show()

In [25]:
gammas = [1e-2]
#lambdas = [1e-1, 1e-3, 10, 100, 1e-5]
lambdas = [1e-1]
train_recalls = []
train_scores = []
train_cdfs = []
test_recalls = []
test_scores = []
test_cdfs = []

m = int(1e6) # number of unknown samples
k = m//10 # number of binding samples (test set size is 118262, k is just some limit we set)
recall_train_samples = get_xy(k) #use for eval

In [ ]:
for g in range(len(gammas)):
    for l in range(len(lambdas)):
        model = SimpleConvNet()
        model.apply(weights_init)
        model.cuda()
    
        print("=============Training=======================")
        sgd(t=1, gamma=gammas[g], lamb=lambdas[l])
        
        print("=============Evaluating train===============")
        train_recall, train_recall_outputs = recall_train(k)
        print("Gamma: ", "%.5f" % gammas[g], "Lambda: ", "%.5f" % lambdas[l], \
              "Train recall: ", "%.2f" % train_recall)
        
        print("=============Evaluating test================")
        test_recall, test_recall_outputs = recall_test(k)
        print("Gamma: ", "%.5f" % gammas[g], "Lambda: ", "%.5f" % lambdas[l], \
              "Test recall: ", "%.2f" % test_recall)
        
        print("=============Evaluating unknown=============")
        eval_unknown_outputs = eval_unknown(m)
        
        train_score = np.asarray(eval_unknown_outputs + train_recall_outputs)
        train_scores.append(train_score)
        
        test_score = np.asarray(eval_unknown_outputs + test_recall_outputs)
        test_scores.append(test_score)
        
        train_cdf = np.sum(np.cumsum(train_score), dtype=float)/(np.sum(train_score)*len(train_score))
        test_cdf = np.sum(np.cumsum(test_score), dtype=float)/(np.sum(test_score)*len(test_score))
        print("G: ", "%.5f" % gammas[g], "L: ", "%.5f" % lambdas[l], \
              "Train CDF: ", "%.3f" % train_cdf, "Test CDF: ", "%.3f" % test_cdf)
        
        train_recalls.append(train_recall)
        test_recalls.append(test_recall)
        train_cdfs.append((gammas[g], lambdas[l], train_cdf))
        test_cdfs.append((gammas[g], lambdas[l],  test_cdf))

print("Train CDFs: ", train_cdfs)
print("Test CDFs: ", test_cdfs)








  0%|          | 0/473047 [00:00<?, ?it/s]

=============Training=======================









  0%|          | 1/473047 [01:00<7930:29:58, 60.35s/it]






  0%|          | 4/473047 [01:00<5552:42:20, 42.26s/it]

Iteration:  0  Loss: 0.787









  0%|          | 7/473047 [01:00<3888:50:54, 29.60s/it]






  0%|          | 10/473047 [01:00<2724:04:29, 20.73s/it]






  0%|          | 12/473047 [01:00<1909:00:38, 14.53s/it]






  0%|          | 14/473047 [01:00<1338:52:48, 10.19s/it]






  0%|          | 17/473047 [01:01<939:05:20,  7.15s/it] 






  0%|          | 19/473047 [01:01<659:25:00,  5.02s/it]






  0%|          | 21/473047 [01:01<463:40:16,  3.53s/it]






  0%|          | 24/473047 [01:01<326:33:51,  2.49s/it]






  0%|          | 26/473047 [01:01<231:02:26,  1.76s/it]






  0%|          | 28/473047 [01:01<164:06:02,  1.25s/it]






  0%|          | 30/473047 [01:01<116:58:54,  1.12it/s]






  0%|          | 32/473047 [01:01<83:54:47,  1.57it/s] 






  0%|          | 34/473047 [01:02<61:00:44,  2.15it/s]






  0%|          | 36/473047 [01:02<44:50:58,  2.93it/s]






  0%|          | 39/473047 [01:02<33:23:47,  3.93it/s]






  0%|          | 41/473047 [01:02<25:28:12,  5.16it/s]






 

...








  1%|          | 3007/473047 [09:32<1196:01:48,  9.16s/it]






  1%|          | 3010/473047 [09:33<838:43:42,  6.42s/it] 






  1%|          | 3012/473047 [09:33<589:13:02,  4.51s/it]






  1%|          | 3015/473047 [09:33<414:26:21,  3.17s/it]






  1%|          | 3018/473047 [09:33<291:26:06,  2.23s/it]






  1%|          | 3021/473047 [09:33<205:36:38,  1.57s/it]






  1%|          | 3024/473047 [09:33<145:56:32,  1.12s/it]






  1%|          | 3027/473047 [09:33<104:21:00,  1.25it/s]






  1%|          | 3030/473047 [09:34<75:58:55,  1.72it/s] 






  1%|          | 3032/473047 [09:34<55:25:11,  2.36it/s]






  1%|          | 3034/473047 [09:34<41:21:27,  3.16it/s]






  1%|          | 3036/473047 [09:34<31:29:43,  4.15it/s]






  1%|          | 3039/473047 [09:34<23:44:19,  5.50it/s]






  1%|          | 3041/473047 [09:34<19:20:42,  6.75it/s]






  1%|          | 3043/473047 [09:34<15:53:25,  8.22it/s]






  1%|          | 3049/473047 [09:34<1

  1%|          | 3288/473047 [09:46<7:02:34, 18.53it/s]






  1%|          | 3290/473047 [09:46<8:30:08, 15.35it/s]






  1%|          | 3292/473047 [09:46<8:15:20, 15.81it/s]






  1%|          | 3294/473047 [09:46<7:52:24, 16.57it/s]






  1%|          | 3296/473047 [09:46<7:48:18, 16.72it/s]






  1%|          | 3299/473047 [09:46<7:07:38, 18.31it/s]






  1%|          | 3302/473047 [09:47<6:49:08, 19.14it/s]






  1%|          | 3304/473047 [09:47<6:56:00, 18.82it/s]






  1%|          | 3306/473047 [09:47<6:55:20, 18.85it/s]






  1%|          | 3308/473047 [09:47<6:56:39, 18.79it/s]






  1%|          | 3311/473047 [09:47<6:44:35, 19.35it/s]






  1%|          | 3314/473047 [09:47<6:42:06, 19.47it/s]






  1%|          | 3316/473047 [09:47<6:56:55, 18.78it/s]






  1%|          | 3319/473047 [09:47<6:37:34, 19.69it/s]






  1%|          | 3322/473047 [09:48<6:12:06, 21.04it/s]






  1%|          | 3325/473047 [09:48<6:17:03, 20.76it/s]






  1%|   

Iteration:  3500  Loss: 0.100









  1%|          | 3506/473047 [10:59<601:23:54,  4.61s/it]






  1%|          | 3508/473047 [10:59<422:57:21,  3.24s/it]






  1%|          | 3511/473047 [10:59<297:46:48,  2.28s/it]






  1%|          | 3514/473047 [10:59<210:13:04,  1.61s/it]






  1%|          | 3517/473047 [10:59<148:59:14,  1.14s/it]






  1%|          | 3520/473047 [10:59<106:06:24,  1.23it/s]






  1%|          | 3523/473047 [11:00<76:01:19,  1.72it/s] 






  1%|          | 3526/473047 [11:00<54:58:12,  2.37it/s]






  1%|          | 3529/473047 [11:00<40:31:32,  3.22it/s]






  1%|          | 3532/473047 [11:00<29:52:50,  4.36it/s]






  1%|          | 3535/473047 [11:00<22:43:56,  5.74it/s]






  1%|          | 3538/473047 [11:00<17:35:07,  7.42it/s]






  1%|          | 3541/473047 [11:00<14:11:20,  9.19it/s]






  1%|          | 3544/473047 [11:00<11:41:36, 11.15it/s]






  1%|          | 3547/473047 [11:01<9:54:35, 13.16it/s] 






  1%|          | 3551/473047 [11:01<8:18:

  1%|          | 3795/473047 [11:12<6:49:23, 19.10it/s]






  1%|          | 3798/473047 [11:12<6:25:14, 20.30it/s]






  1%|          | 3801/473047 [11:13<6:23:53, 20.37it/s]






  1%|          | 3804/473047 [11:13<6:38:42, 19.62it/s]






  1%|          | 3806/473047 [11:13<7:37:45, 17.08it/s]






  1%|          | 3808/473047 [11:13<7:29:29, 17.40it/s]






  1%|          | 3810/473047 [11:13<7:15:20, 17.96it/s]






  1%|          | 3812/473047 [11:13<7:09:09, 18.22it/s]






  1%|          | 3815/473047 [11:13<6:46:04, 19.26it/s]






  1%|          | 3817/473047 [11:13<6:41:56, 19.46it/s]






  1%|          | 3820/473047 [11:14<6:55:30, 18.82it/s]






  1%|          | 3823/473047 [11:14<6:40:43, 19.52it/s]






  1%|          | 3826/473047 [11:14<6:10:59, 21.08it/s]






  1%|          | 3829/473047 [11:14<6:18:52, 20.64it/s]






  1%|          | 3832/473047 [11:14<6:48:53, 19.13it/s]






  1%|          | 3834/473047 [11:14<6:55:52, 18.80it/s]






  1%|   

In [ ]:
cdf(train_scores, 0)